### Código Compartilhado
> Notebook utilizado para código que é compartilhado por diferentes notebooks

* Import das bibliotecas mais utilizadas

In [3]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from math import floor, ceil

from IPython.display import Audio


import sys  
sys.path.insert(0, '/Users/gustavofigueiredo/projects/ml-beat-box/audio_classify/src/')
from mel_features import log_mel_spectrogram

* Contantes

In [1]:
LABEL = ["music","not_music"]
MODEL_PATH = '../model/'
SAMPLE_RATE = 22050
HOP_LENGTH = 512
FRAME_LENGTH = 2048
BPM_RATE = 120
F_MFCC = 13
SEGMENT_DURATION = 3
N_MELS=32
SAMPLES_PER_SEGMENT = SAMPLE_RATE * SEGMENT_DURATION

* Algumas funções que são reutilizáveis 

> Código que encapsula o gráfico do formato da onda de um sinal enviado.

In [ ]:
def my_wave_plot(audio):
    plt.figure(figsize=(12, 4))
    librosa.display.waveplot(audio, sr=SAMPLE_RATE)
    plt.show()

> Código que serve para geração das fetuares baseadas em croma e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_chroma_plot(audio):
    chromagram = librosa.feature.chroma_cqt(y=audio, sr=SAMPLE_RATE, hop_length=HOP_LENGTH)
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=HOP_LENGTH, cmap='coolwarm')
    plt.show()

> Código que serve para geração das fetuares baseadas em energia e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_energy_plot(audio):
    rms = librosa.feature.rms(y=audio, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH, center=True)
    rms = rms[0]
    energy = np.array([sum(abs(audio[i:i+FRAME_LENGTH]**2)) for i in range(0, len(audio+1), HOP_LENGTH) ])
    frames = range(len(energy))
    plt.figure(figsize=(12, 4))
    t = librosa.frames_to_time(frames, sr=SAMPLE_RATE, hop_length=HOP_LENGTH)
    librosa.display.waveplot(audio, sr=SAMPLE_RATE, alpha=0.4)
    plt.plot(t, energy/energy.max(), 'r--')             # normalized for visualization
    frames = range(len(rms))
    t = librosa.frames_to_time(frames, sr=SAMPLE_RATE, hop_length=HOP_LENGTH)
    plt.plot(t, rms/rms.max(), color='g') # normalized for visualization
    plt.legend(('Energy', 'RMSE'))
    plt.show()

> Código que serve para geração das fetuares baseadas em croma e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_beat_track_plot(audio):
    onset_env = librosa.onset.onset_strength(y=audio, sr=SAMPLE_RATE)
    tempo, beats = librosa.beat.beat_track(onset_envelope=onset_env)
    times = librosa.times_like(onset_env, sr=SAMPLE_RATE)
    plt.figure(figsize=(12, 4))
    plt.plot(times, librosa.util.normalize(onset_env), label='Onset strength')
    plt.vlines(times[beats], 0, 1, alpha=0.5, color='r', linestyle='--', label='Beats')
    plt.legend()
    plt.show()

> Código que serve para geração das fetuares baseadas em plp e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_plp_plot(audio):
    onset_env = librosa.onset.onset_strength(y=audio, sr=SAMPLE_RATE)
    pulse = librosa.beat.plp(onset_envelope=onset_env, sr=SAMPLE_RATE)
    beats_plp = np.flatnonzero(librosa.util.localmax(pulse))
    times = librosa.times_like(pulse, sr=SAMPLE_RATE)
    plt.figure(figsize=(12, 4))
    plt.plot(times, librosa.util.normalize(pulse), label='PLP')
    plt.vlines(times[beats_plp], 0, 1, alpha=0.5, color='r', linestyle='--', label='PLP Beats')
    plt.legend()
    plt.show()

> Código que serve para geração das fetuares baseadas em MFCC e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_mfcc_plot(audio):
    from sklearn.preprocessing import MinMaxScaler
    plt.figure(figsize=(12, 4))
    mfccs = librosa.feature.mfcc(audio, sr=SAMPLE_RATE)
    scaler = MinMaxScaler()
    mfccs = scaler.fit_transform(mfccs)
    librosa.display.specshow(mfccs, sr=SAMPLE_RATE, x_axis='time')
    plt.show()

> Código que serve para geração das fetuares baseadas energia e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_novelty_energy_plot(audio):
    rmse = librosa.feature.rms(audio, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH).flatten()
    rmse_diff = np.zeros_like(rmse)
    rmse_diff[1:] = np.diff(rmse)
    energy_novelty = np.max([np.zeros_like(rmse_diff), rmse_diff], axis=0)
    frames = np.arange(len(rmse))
    t = librosa.frames_to_time(frames, sr=SAMPLE_RATE)
    plt.figure(figsize=(15, 6))
    plt.plot(t, rmse, 'b--', t, rmse_diff, 'g--^', t, energy_novelty, 'r-')
    plt.xlim(0, t.max())
    plt.xlabel('Time (sec)')
    plt.legend(('RMSE', 'delta RMSE', 'energy novelty')) 

> Código que serve para geração das fetuares baseado no spectral centroid e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

def my_spectral_centroid_plot(audio):
    spectral_centroids = librosa.feature.spectral_centroid(audio+0.01, sr=SAMPLE_RATE)[0]
    frames = range(len(spectral_centroids))
    t = librosa.frames_to_time(frames)
    plt.figure(figsize=(15, 6))
    librosa.display.waveplot(audio, sr=SAMPLE_RATE, alpha=0.4)
    plt.plot(t, normalize(spectral_centroids), color='r') # normalize for visualization purposes
    plt.show()

> Código que serve para geração das fetuares baseadas em zero crossing rate e ao mesmo tempo gerar o gráfico para permitir sua visualização.

In [ ]:
def my_zero_cross_plot(audio):
    zcrs = librosa.feature.zero_crossing_rate(audio + 0.0001)[0]
    zcrs_diff = np.zeros_like(zcrs)
    zcrs_diff[1:] = np.diff(zcrs)
    plt.figure(figsize=(14, 5))
    plt.plot(zcrs)
    plt.plot(zcrs_diff)
    plt.show()

> Código para converter a entrada no formato da entrada do modelo

In [7]:
def _generate_mfcc(filtered_signal):
    mfcc = librosa.feature.mfcc(filtered_signal, 
                                sr = SAMPLE_RATE,
                                n_mfcc=F_MFCC,
                                n_fft=FRAME_LENGTH,
                                hop_length=HOP_LENGTH)
    return mfcc

def _generate_rms(filtered_signal):
    rms = librosa.feature.rms(y=filtered_signal, 
                              frame_length=SAMPLE_RATE, 
                              hop_length=HOP_LENGTH, 
                              center=True)
    return rms

def _generate_spectral_centroid(filtered_signal):
    centroid = librosa.feature.spectral_centroid(filtered_signal+0.01, 
                                                 sr=SAMPLE_RATE)
    return centroid

def _generate_zero_crossing_rate(filtered_signal):
    zcr = librosa.feature.zero_crossing_rate(y=filtered_signal + 0.01)
    return zcr

def _generate_onset_strength(filtered_signal):
    onset = librosa.onset.onset_strength(y=filtered_signal, 
        sr=SAMPLE_RATE)
    onset = onset[np.newaxis, ...]
    return onset

def _generate_mel_spectogram(filtered_signal, version):
    if version == "normal":
        mel = librosa.feature.melspectrogram(y=filtered_signal, 
                                             sr=SAMPLE_RATE, 
                                             n_mels=N_MELS, 
                                             fmax=10000 , 
                                             n_fft=FRAME_LENGTH, 
                                             hop_length=HOP_LENGTH)
    else:
        mel = log_mel_spectrogram(filtered_signal, audio_sample_rate=SAMPLE_RATE, log_offset=0.01)
    return mel

def generate_input(signal, list_of_features, version):
    num_segments = floor(len(signal)/SAMPLES_PER_SEGMENT)
    samples = []
    
    # process all segments of audio file
    for d in range(num_segments):
        # calculate start and finish sample for current segment
        start = SAMPLES_PER_SEGMENT * d
        finish = start + SAMPLES_PER_SEGMENT
        filtered_signal =  signal[start:finish]
        
        features = {
            "centroid": _generate_spectral_centroid(filtered_signal),
            "mfcc": _generate_mfcc(filtered_signal),
            "mel_spec": _generate_mel_spectogram(filtered_signal, version),
            "onset": _generate_onset_strength(filtered_signal),
            "rms": _generate_rms(filtered_signal),
            "zcr": _generate_zero_crossing_rate(filtered_signal)
        }
        
        f1 = features[list_of_features[0]]
        for i in range(1, len(list_of_features)):
            f_other = features[list_of_features[i]]
            f1 = np.concatenate([f1, f_other], axis = 0)
        samples.append(f1.T)
    samples = np.array(samples) 
    return samples

> Código para rodar o modelo treinado de acordo com as features selecionadas.

In [ ]:
def run_model(features, test_file, suffix=[], version="normal"):
    model = keras.models.load_model('../model/mlp_beat_box_{}'.format("_".join(features + suffix)))
    intro_signal, _ = librosa.load(test_file, sr=SAMPLE_RATE)
    f_input = generate_input(intro_signal, features, version)
    X = f_input[..., np.newaxis] 
    import warnings
    print("SAMPLE SHAPE: {}".format(X.shape))
    warnings.filterwarnings('ignore')
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    pLabel = list(map(lambda x: LABEL[x] ,predicted_index))
    plt.figure(figsize=(12, 4))
    plt.plot(pLabel)
    plt.show()
    del model